## 이 실습자료에서는 간단한 분류 문제에 대해 Naive Bayes를 학습, 평가하는 과정을 합니다.

### 활용할 데이터
- <span style = 'font-size:1.2em;line-height:1.5em'>scikit-learn에서 기본으로 제공하는 데이터 중, iris data</span>


In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris


### 데이터 불러오고 확인해보기

In [2]:
iris = load_iris()

In [3]:
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [4]:
iris.data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [5]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [6]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [7]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [8]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [9]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [10]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

### 데이터 준비하기
- <span style = 'font-size:1.2em;line-height:1.5em'>여기선 별다른 전처리 과정은 없고, 단순히 X,y 데이터 통합 및 train-test split만 합니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>(굳이 이와 같이 데이터 통합을 꼭 해야할 필요는 없습니다.)</span>

In [12]:
x = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.DataFrame(iris.target, columns = ['target'])

In [13]:
df = pd.concat((x,y), axis=1)

In [14]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB # Need to conduct Naive Bayes modeling for multi-class data
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

- <span style = 'font-size:1.2em;line-height:1.5em'>train-test split하기</span>

In [16]:
df_trn, df_tst = train_test_split(df, test_size=0.2, stratify=df['target'])

In [17]:
print(df_trn.shape[0], df_tst.shape[0])

120 30


### 모델 학습 및 평가하기

- <span style = 'font-size:1.2em;line-height:1.5em'>Cross-validation으로 모델의 CV score 측정하기 (여기서는 accuracy)</span>

In [20]:
model = MultinomialNB()
cv_scores = cross_val_score(model, X=df_trn.iloc[:,:-1], y=df_trn.iloc[:,-1], cv=5, scoring='accuracy')
print(f"평균 accuracy: {np.mean(cv_scores):.3f}")

평균 accuracy: 0.950


- <span style = 'font-size:1.2em;line-height:1.5em'>모델을 학습하고 test set에서의 성능을 측정하기 (generalization performance의 추정치)</span>

In [23]:
# 모델 학습하기
model.fit(X=df_trn.iloc[:,:-1], y=df_trn.iloc[:,-1])

# test set에서 model의 성능 측정하기
y_pred = model.predict(df_tst.iloc[:,:-1])
y_true = df_tst.iloc[:,-1]

acc = accuracy_score(y_true=y_true, y_pred=y_pred)
pre = precision_score(y_true=y_true, y_pred=y_pred, average='macro')
rec = recall_score(y_true=y_true, y_pred=y_pred, average='macro')
f1 = f1_score(y_true=y_true, y_pred=y_pred, average='macro')

print(f"Performance: accuracy={acc:.3f}, precision={pre:.3f}, recall={rec:.3f}, f1={f1:.3f}")

Performance: accuracy=0.967, precision=0.970, recall=0.967, f1=0.967


- <span style = 'font-size:1.2em;line-height:1.5em'>test set에서의 confusion matrix 산출하기</span>

In [24]:
mat = confusion_matrix(y_true=y_true, y_pred=y_pred)

In [25]:
print(mat)

[[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]
